# Introduction to neural nets

## 2. Multi-layer perceptron
<figure align="middle">
    <img src="images/perceptron_2layers.png" title="2-layer perceptron" width="400px">
    <figcaption>Fig 1. Multi-layer perceptron with 1 hidden layer.</figcaption>
</figure>

Implementation of a 2-layer perceptron: input layer plus hidden layer plus output.

In [1]:
import numpy as np

class MLPerceptron:
    """ A Multi-Layer Perceptron"""

    def __init__(self, inputs, targets, nhidden, beta=1, momentum=0.9, outtype='logistic'):
        """ Constructor """
        # Set up network size
        self.nin = np.shape(inputs)[1]
        self.nout = np.shape(targets)[1]
        self.ndata = np.shape(inputs)[0]
        self.nhidden = nhidden

        self.beta = beta
        self.momentum = momentum
        self.outtype = outtype

        # Initialise network
        self.weights1 = (np.random.rand(self.nin + 1, self.nhidden) - 0.5) * 2 / np.sqrt(self.nin)
        self.weights2 = (np.random.rand(self.nhidden + 1, self.nout) - 0.5) * 2 / np.sqrt(self.nhidden)



    def train(self, inputs, targets, eta, niterations):
        """ Train the thing """
        # Add the bias node
        inputs_with_bias = np.concatenate((inputs, -np.ones((self.ndata, 1))), axis=1)
        change = range(self.ndata)

        updatew1 = np.zeros((np.shape(self.weights1)))
        updatew2 = np.zeros((np.shape(self.weights2)))

        current_iteration = 0
        for n in range(niterations):
            current_iteration += 1
            self.outputs = self.forward(inputs_with_bias)

            error = 0.5 * np.sum((self.outputs - targets) ** 2)
            
            if (np.mod(n, 50) == 0):
                print("Iteration: ", n+1, " Error: ", error)

            
            # Compute delta between targets and outputs
            # Different types of output neurons
            if self.outtype == 'linear':
                deltao = (self.outputs - targets) / self.ndata
            elif self.outtype == 'logistic':
                deltao = self.beta * (self.outputs - targets) * self.outputs * (1.0 - self.outputs)
            elif self.outtype == 'softmax':
                deltao = (self.outputs - targets) * (self.outputs * (-self.outputs) + self.outputs) / self.ndata
            else:
                print("error")

            
            # Compute delta for each weight from input to hidden
            deltah = self.hidden * self.beta * (1.0 - self.hidden) * (np.dot(deltao, np.transpose(self.weights2)))

            # backpropagation of error
            updatew1 = eta * (np.dot(np.transpose(inputs_with_bias), deltah[:, :-1])) + self.momentum * updatew1
            updatew2 = eta * (np.dot(np.transpose(self.hidden), deltao)) + self.momentum * updatew2
            self.weights1 -= updatew1
            self.weights2 -= updatew2

            cm, accuracy = self.confusion_matrix(inputs, targets)
            if accuracy == 1.0:
                break
        print("Total {} iterations".format(current_iteration))

    def forward(self, inputs):
        """ Run the network forward """

        self.hidden = np.dot(inputs, self.weights1);
        self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))
        self.hidden = np.concatenate((self.hidden, -np.ones((np.shape(inputs)[0], 1))), axis=1)

        outputs = np.dot(self.hidden, self.weights2);

        # Different types of output neurons
        if self.outtype == 'linear':
            return outputs
        elif self.outtype == 'logistic':
            return 1.0 / (1.0 + np.exp(-self.beta * outputs))
        elif self.outtype == 'softmax':
            normalisers = np.sum(np.exp(outputs), axis=1) * np.ones((1, np.shape(outputs)[0]))
            return np.transpose(np.transpose(np.exp(outputs)) / normalisers)
        else:
            print("error")

    def confusion_matrix(self, inputs, targets):
        """Confusion matrix"""

        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs, -np.ones((np.shape(inputs)[0], 1))), axis=1)
        outputs = self.forward(inputs)

        nclasses = np.shape(targets)[1]

        if nclasses == 1:
            nclasses = 2
            outputs = np.where(outputs > 0.5, 1, 0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs, 1)
            targets = np.argmax(targets, 1)

        cm = np.zeros((nclasses, nclasses))
        for i in range(nclasses):
            for j in range(nclasses):
                cm[i, j] = np.sum(np.where(outputs == i, 1, 0) * np.where(targets == j, 1, 0))


        return cm, np.trace(cm) / np.sum(cm)

## 2.1. Learning AND
This still learns a linear separator

In [ ]:
anddata = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1]])

p = MLPerceptron(anddata[:, 0:2], anddata[:, 2:3], 2)
p.train(anddata[:, 0:2], anddata[:, 2:3], 0.25, 1001)

print()
cm, accuracy = p.confusion_matrix(anddata[:, 0:2], anddata[:, 2:3])
print("Confusion matrix:")
print(cm)
print("Accuracy: ", accuracy)

## 2.2. Learning XOR

In [ ]:
xordata = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]])
q = MLPerceptron(xordata[:, 0:2], xordata[:, 2:3], 2, outtype='logistic')
q.train(xordata[:, 0:2], xordata[:, 2:3], 0.25, 5001)

print()
cm, accuracy = q.confusion_matrix(xordata[:, 0:2], xordata[:, 2:3])
print("Confusion matrix:")
print(cm)
print("Accuracy: ",accuracy)

<figure align="middle">
    <img src="images/mlp_XOR.png" title="Separated XOR by hyperplane" width="400px">
    <figcaption>Fig 2. Separated the linearly non-separable XOR by a hyperplane.</figcaption>
</figure>

## 2.3. Breast cancer diagnosis

The simple 2-layer perceptron implemented above is quite powerful.
We are going to use it to classify mammography results into malignant and non-malignant. 
The dataset is from [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).

In [6]:
data_file = "wdbc.csv"

In [7]:
import csv

def csv_to_array(file_name):
    np_a = None
    with open(file_name) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            class_label = 0
            if row[0].strip()=='M':
                class_label = 1
            num_arr = [float(x) for x in row[1:]] + [class_label]

            if np_a is None:
                np_a = np.array(num_arr)
            else:
                np_a = np.c_[np_a, num_arr]

    np_a = np.transpose(np_a)

    return np_a

In [8]:
bc_data = csv_to_array(data_file)
n_rows = np.shape(bc_data)[0]
n_cols = np.shape(bc_data)[1]
print("Dataset contains",n_rows,"rows and", n_cols,"columns")

# keep target attribute 0 or 1
target_a = bc_data[:,n_cols-1:n_cols ]

# normalize data in each column by subtracting mean and dividing by variance
bc_data = bc_data[:, 0:n_cols-1]
bc_data = (bc_data - bc_data.mean(axis=0)) / bc_data.var(axis=0)

bc_data = np.c_[bc_data,target_a]

Dataset contains 569 rows and 31 columns


In [11]:
train_set = bc_data[0:3*n_rows//4]
test_set = bc_data[3*n_rows//4:]

n_rows = np.shape(train_set)[0]
n_cols = np.shape(train_set)[1]
print("Train set contains",n_rows,"rows and", n_cols,"columns")

n_rows = np.shape(test_set)[0]
n_cols = np.shape(test_set)[1]
print("Test set contains",n_rows,"rows and", n_cols,"columns")

Train set contains 426 rows and 31 columns
Test set contains 143 rows and 31 columns


Train the network:

In [21]:
p = MLPerceptron(train_set[:, 0:n_cols-1], train_set[:, n_cols-1:n_cols], 2, outtype="linear")
p.train(train_set[:, 0:n_cols-1],  train_set[:, n_cols-1:n_cols], 0.0005, 1000001)

Iteration:  1  Error:  56.387409328029456
Iteration:  51  Error:  54.74560818459395
Iteration:  101  Error:  53.292998519235155
Iteration:  151  Error:  52.70732820528875
Iteration:  201  Error:  52.26662673591269
Iteration:  251  Error:  51.6864549345454
Iteration:  301  Error:  51.262546722980375
Iteration:  351  Error:  50.80306974659918
Iteration:  401  Error:  50.39202636581441
Iteration:  451  Error:  49.714021698233196
Iteration:  501  Error:  48.95792945616171
Iteration:  551  Error:  48.20365303245444
Iteration:  601  Error:  47.48297386282173
Iteration:  651  Error:  46.97072340915628
Iteration:  701  Error:  46.479704992574746
Iteration:  751  Error:  46.00920199781628
Iteration:  801  Error:  43.19261619525064
Iteration:  851  Error:  39.86461810746873
Iteration:  901  Error:  38.650401737241936
Iteration:  951  Error:  37.456662227771105
Iteration:  1001  Error:  36.35129980926143
Iteration:  1051  Error:  35.43879325608947
Iteration:  1101  Error:  34.60160772830109
Itera

Iteration:  9751  Error:  7.876544196195928
Iteration:  9801  Error:  7.869408905500782
Iteration:  9851  Error:  7.862293642741353
Iteration:  9901  Error:  7.85519619504316
Iteration:  9951  Error:  7.848114358421414
Iteration:  10001  Error:  7.841045940548939
Iteration:  10051  Error:  7.833988765028197
Iteration:  10101  Error:  7.8269406770708585
Iteration:  10151  Error:  7.819899550419708
Iteration:  10201  Error:  7.8128632952852115
Iteration:  10251  Error:  7.805829867014188
Iteration:  10301  Error:  7.7987972751568835
Iteration:  10351  Error:  7.791763592544008
Iteration:  10401  Error:  7.7847269639187875
Iteration:  10451  Error:  7.777685613585782
Iteration:  10501  Error:  7.7706378514403465
Iteration:  10551  Error:  7.763582076639459
Iteration:  10601  Error:  7.756516778078231
Iteration:  10651  Error:  7.749440530747135
Iteration:  10701  Error:  7.742351986926451
Iteration:  10751  Error:  7.735249860902701
Iteration:  10801  Error:  7.728132905157841
Iteration: 

Iteration:  19051  Error:  6.190880983274171
Iteration:  19101  Error:  6.183824223040782
Iteration:  19151  Error:  6.1767698626012795
Iteration:  19201  Error:  6.169713752508134
Iteration:  19251  Error:  6.162651382572195
Iteration:  19301  Error:  6.155577829916972
Iteration:  19351  Error:  6.148487698164766
Iteration:  19401  Error:  6.141375045974916
Iteration:  19451  Error:  6.134233302758556
Iteration:  19501  Error:  6.1270551689077
Iteration:  19551  Error:  6.119832497280805
Iteration:  19601  Error:  6.112556151963473
Iteration:  19651  Error:  6.105215839455104
Iteration:  19701  Error:  6.097799906412579
Iteration:  19751  Error:  6.090295096924486
Iteration:  19801  Error:  6.082686261053431
Iteration:  19851  Error:  6.074956005218073
Iteration:  19901  Error:  6.067084274207071
Iteration:  19951  Error:  6.0590478548465745
Iteration:  20001  Error:  6.050819793748563
Iteration:  20051  Error:  6.042368728259322
Iteration:  20101  Error:  6.033658144391994
Iteration:

Iteration:  28401  Error:  5.173228895335926
Iteration:  28451  Error:  5.171159379715238
Iteration:  28501  Error:  5.169101073260658
Iteration:  28551  Error:  5.167053864745171
Iteration:  28601  Error:  5.165017644512164
Iteration:  28651  Error:  5.162992304444902
Iteration:  28701  Error:  5.160977737936806
Iteration:  28751  Error:  5.158973839862532
Iteration:  28801  Error:  5.156980506549805
Iteration:  28851  Error:  5.154997635751986
Iteration:  28901  Error:  5.153025126621365
Iteration:  28951  Error:  5.151062879683118
Iteration:  29001  Error:  5.149110796809945
Iteration:  29051  Error:  5.14716878119734
Iteration:  29101  Error:  5.145236737339481
Iteration:  29151  Error:  5.143314571005725
Iteration:  29201  Error:  5.141402189217674
Iteration:  29251  Error:  5.139499500226812
Iteration:  29301  Error:  5.137606413492666
Iteration:  29351  Error:  5.135722839661518
Iteration:  29401  Error:  5.1338486905456024
Iteration:  29451  Error:  5.131983879102799
Iteration:

Iteration:  37651  Error:  4.910009688759545
Iteration:  37701  Error:  4.90899625882251
Iteration:  37751  Error:  4.907985564717517
Iteration:  37801  Error:  4.906977590313542
Iteration:  37851  Error:  4.905972319668225
Iteration:  37901  Error:  4.904969737027772
Iteration:  37951  Error:  4.903969826826812
Iteration:  38001  Error:  4.902972573688142
Iteration:  38051  Error:  4.901977962422391
Iteration:  38101  Error:  4.900985978027597
Iteration:  38151  Error:  4.899996605688673
Iteration:  38201  Error:  4.899009830776775
Iteration:  38251  Error:  4.898025638848557
Iteration:  38301  Error:  4.897044015645305
Iteration:  38351  Error:  4.896064947091963
Iteration:  38401  Error:  4.895088419296037
Iteration:  38451  Error:  4.894114418546363
Iteration:  38501  Error:  4.893142931311761
Iteration:  38551  Error:  4.892173944239563
Iteration:  38601  Error:  4.891207444154007
Iteration:  38651  Error:  4.890243418054507
Iteration:  38701  Error:  4.889281853113799
Iteration: 

...

Iteration:  961901  Error:  3.102606320722196
Iteration:  961951  Error:  3.10258229652848
Iteration:  962001  Error:  3.1025582741068374
Iteration:  962051  Error:  3.102534253457056
Iteration:  962101  Error:  3.1025102345789257
Iteration:  962151  Error:  3.102486217472233
Iteration:  962201  Error:  3.1024622021367687
Iteration:  962251  Error:  3.10243818857232
Iteration:  962301  Error:  3.1024141767786766
Iteration:  962351  Error:  3.102390166755624
Iteration:  962401  Error:  3.1023661585029547
Iteration:  962451  Error:  3.102342152020455
Iteration:  962501  Error:  3.1023181473079156
Iteration:  962551  Error:  3.1022941443651226
Iteration:  962601  Error:  3.1022701431918653
Iteration:  962651  Error:  3.1022461437879363
Iteration:  962701  Error:  3.1022221461531214
Iteration:  962751  Error:  3.102198150287208
Iteration:  962801  Error:  3.102174156189988
Iteration:  962851  Error:  3.102150163861249
Iteration:  962901  Error:  3.1021261733007806
Iteration:  962951  Error

Iteration:  970701  Error:  3.098405164933465
Iteration:  970751  Error:  3.0983814493977215
Iteration:  970801  Error:  3.0983577355973533
Iteration:  970851  Error:  3.0983340235321544
Iteration:  970901  Error:  3.09831031320192
Iteration:  970951  Error:  3.098286604606445
Iteration:  971001  Error:  3.098262897745525
Iteration:  971051  Error:  3.098239192618952
Iteration:  971101  Error:  3.098215489226522
Iteration:  971151  Error:  3.098191787568033
Iteration:  971201  Error:  3.0981680876432747
Iteration:  971251  Error:  3.098144389452045
Iteration:  971301  Error:  3.098120692994137
Iteration:  971351  Error:  3.0980969982693467
Iteration:  971401  Error:  3.0980733052774694
Iteration:  971451  Error:  3.0980496140183007
Iteration:  971501  Error:  3.098025924491633
Iteration:  971551  Error:  3.0980022366972633
Iteration:  971601  Error:  3.097978550634984
Iteration:  971651  Error:  3.097954866304595
Iteration:  971701  Error:  3.097931183705886
Iteration:  971751  Error: 

Iteration:  979551  Error:  3.09423436000024
Iteration:  979601  Error:  3.0942109484569396
Iteration:  979651  Error:  3.0941875386131796
Iteration:  979701  Error:  3.094164130468763
Iteration:  979751  Error:  3.094140724023487
Iteration:  979801  Error:  3.094117319277154
Iteration:  979851  Error:  3.0940939162295615
Iteration:  979901  Error:  3.0940705148805154
Iteration:  979951  Error:  3.094047115229814
Iteration:  980001  Error:  3.0940237172772562
Iteration:  980051  Error:  3.0940003210226443
Iteration:  980101  Error:  3.0939769264657797
Iteration:  980151  Error:  3.093953533606463
Iteration:  980201  Error:  3.0939301424444947
Iteration:  980251  Error:  3.0939067529796747
Iteration:  980301  Error:  3.0938833652118056
Iteration:  980351  Error:  3.0938599791406873
Iteration:  980401  Error:  3.0938365947661213
Iteration:  980451  Error:  3.093813212087909
Iteration:  980501  Error:  3.09378983110585
Iteration:  980551  Error:  3.0937664518197474
Iteration:  980601  Err

Iteration:  988851  Error:  3.0899088446340297
Iteration:  988901  Error:  3.0898857458078757
Iteration:  988951  Error:  3.0898626486447207
Iteration:  989001  Error:  3.0898395531443734
Iteration:  989051  Error:  3.089816459306638
Iteration:  989101  Error:  3.089793367131321
Iteration:  989151  Error:  3.0897702766182307
Iteration:  989201  Error:  3.089747187767175
Iteration:  989251  Error:  3.089724100577956
Iteration:  989301  Error:  3.0897010150503865
Iteration:  989351  Error:  3.0896779311842666
Iteration:  989401  Error:  3.089654848979409
Iteration:  989451  Error:  3.089631768435617
Iteration:  989501  Error:  3.0896086895526995
Iteration:  989551  Error:  3.0895856123304615
Iteration:  989601  Error:  3.0895625367687125
Iteration:  989651  Error:  3.089539462867255
Iteration:  989701  Error:  3.0895163906259
Iteration:  989751  Error:  3.0894933200444537
Iteration:  989801  Error:  3.0894702511227248
Iteration:  989851  Error:  3.089447183860516
Iteration:  989901  Erro

Iteration:  997701  Error:  3.0858460807721073
Iteration:  997751  Error:  3.0858232733124433
Iteration:  997801  Error:  3.085800467482028
Iteration:  997851  Error:  3.0857776632806737
Iteration:  997901  Error:  3.0857548607081924
Iteration:  997951  Error:  3.085732059764399
Iteration:  998001  Error:  3.085709260449103
Iteration:  998051  Error:  3.0856864627621166
Iteration:  998101  Error:  3.0856636667032507
Iteration:  998151  Error:  3.0856408722723225
Iteration:  998201  Error:  3.08561807946914
Iteration:  998251  Error:  3.0855952882935167
Iteration:  998301  Error:  3.085572498745265
Iteration:  998351  Error:  3.0855497108241976
Iteration:  998401  Error:  3.085526924530127
Iteration:  998451  Error:  3.0855041398628655
Iteration:  998501  Error:  3.085481356822225
Iteration:  998551  Error:  3.085458575408019
Iteration:  998601  Error:  3.0854357956200573
Iteration:  998651  Error:  3.085413017458157
Iteration:  998701  Error:  3.085390240922126
Iteration:  998751  Erro

In [19]:
cm, accuracy = p.confusion_matrix(train_set[:, 0:n_cols-1], train_set[:, n_cols-1:n_cols])
print("Confusion matrix:")
print(cm)
print("Train Accuracy: ",accuracy)

Confusion matrix:
[[242.   5.]
 [  7. 172.]]
Train Accuracy:  0.971830985915493


In [20]:
cm, accuracy = p.confusion_matrix(test_set[:, 0:n_cols-1], test_set[:, n_cols-1:n_cols])
print("Confusion matrix:")
print(cm)
print("Test Accuracy: ",accuracy)

Confusion matrix:
[[103.   1.]
 [  5.  34.]]
Test Accuracy:  0.958041958041958


Copyright &copy; 2020 Marina Barsky. All rights reserved.